# Building the model layers
## Build a neural network
뉴럴 네트워크는 데이터 작업을 수행하는 레이어와 모듈로 구성됩니다. [torch.nn](https://pytorch.org/docs/stable/nn.html) 네임스페이스는 사용자 고유의 신경망을 구성할 수 있는 모든 구성 요소를 제공합니다. PyTorch 모든 모듈은 [nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html)를 상속하며, 네트워크는 다른 모듈(레이어)로 구성된 모듈 자체입니다. 이러한 중첩 구조를 통해 복잡한 네트워크 구조를 쉽게 관리하고 생성할 수 있습니다.

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

### Get a hardware device for training
우리는 GPU 같은 하드웨어 가속기를 통해 모델을 학습 하기를 희망합니다. 이를 위해 [torch.cuda](https://pytorch.org/docs/stable/notes/cuda.html)가 사용 가능한지 확인해봅시다. 불가능하다면 CPU를 사용하여 진행할 수 있습니다.

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


### Define the class

`nn.module`를 상속하여 뉴럴넷을 정의하고 `__init__` 함수에서 레이어를 초기화 합니다. `nn.module`를 상속한 모든 하위 클래스는 입력 데이터의 연산 방법을 `forward` 함수에 구현합니다.

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

`NeuralNetwork` 인스턴스를 생성하여 `device`를 옮기고 뉴럴넷의 구조를 출력합니다.

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


모델을 사용하기 위해 네트워크에 입력 데이터를 통과시키면 백드라운드 연산에 의해 모델의 `forward` 함수가 실행됩니다. `model.forward()`를 직접 명시적으로 호출해서는 안됩니다. 

입력 데이터가 모델을 호출하면 각 클래스를 예측한 초기값의 10 차원의 텐서로 반환됩니다. `nn.softmax 모듈`의 인스턴스를 통과시킴으로써 확률을 얻을 수 있습니다.

In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)

"""
tensor([[0.0983, 0.1032, 0.1035, 0.0983, 0.0983, 0.0983, 0.0983, 0.1008, 0.1027,
         0.0983]], device='cuda:0', grad_fn=<SoftmaxBackward>)
"""

y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([9], device='cuda:0')


### Model layers

FashionMNIST model의 네트워크를 자세히 살펴봅시다. 이것을 표현하기 위해, 이미지 3개(28×28)로 구성된 미니배치를 취하고 네트워크에 통과시켰을 때 무엇이 일어나는지 봅시다.

In [6]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


#### nn.Flatten
[nn.Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html)는 2D 28×28 이미지를 연속된 784픽셀 이미지로 전환하는(미니배치 차원(dim=0)은 유지됨) 모듈입니다.

In [7]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


#### nn.Linear
[linear layer](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)는 저장된 가중치와 편향을 사용하여 입력 데이터에 선형 변환을 적용하는 모듈입니다. 

In [8]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


#### nn.ReLU
비선형 활성 함수는 모델의 입력과 출력 사이의 복잡한 연결(비선형성)을 추가합니다. 선형 변환 후에 적용된 비선형성은 뉴럴넷이 다양한현상을 학습하도록 돕습니다.

이 모델에서는 선형 레이어 사이에서 [nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html)을 사용하였으나, 비선형성을 부여할 수 있는 다른 활성 함수도 존재합니다.

In [9]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.0677,  0.0203, -0.0507,  0.0315, -0.1010,  0.3733, -0.3353,  0.1764,
         -0.0703,  0.0589, -0.0216,  0.2784, -0.1285, -0.1544, -0.5395,  0.7219,
          0.1978,  0.2174,  0.4620, -0.0267],
        [-0.1882,  0.1008,  0.1437, -0.2551, -0.3116,  0.3305, -0.2943,  0.0495,
         -0.1340, -0.2476, -0.0630,  0.1996, -0.4020, -0.3026, -0.3873,  0.3520,
         -0.0305, -0.0571,  0.1514, -0.1033],
        [-0.2485,  0.1090,  0.2022, -0.1402, -0.2839,  0.4608,  0.0945,  0.0838,
         -0.0377, -0.0240, -0.0947,  0.0670, -0.3075, -0.3889, -0.3877,  0.4988,
          0.3697, -0.2467,  0.1534,  0.1560]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.0000, 0.0203, 0.0000, 0.0315, 0.0000, 0.3733, 0.0000, 0.1764, 0.0000,
         0.0589, 0.0000, 0.2784, 0.0000, 0.0000, 0.0000, 0.7219, 0.1978, 0.2174,
         0.4620, 0.0000],
        [0.0000, 0.1008, 0.1437, 0.0000, 0.0000, 0.3305, 0.0000, 0.0495, 0.0000,
         0.0000, 0.0000, 0.1996, 0.0000, 0.0000, 0.000

#### nn.Sequential

[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html)은 모듈의 순서가 정의된 컨테이너입니다. 데이터는 정의된 것과 동일한 순서대로 모든 모듈을 통과하여 전달됩니다. `seq_modules` 같이 빠르게 네트워크를 만들기 위해 nn.Sequential를 사용할 수 있습니다. 

In [10]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)

input_image - torch.rand(3, 28, 28)
logits = seq_modules(input_image)

print(logits)

tensor([[ 4.2472e-02, -1.5664e-02,  3.1470e-02,  1.4924e-01, -7.3051e-02,
         -1.1255e-01,  2.1065e-03,  3.7320e-01,  3.7471e-01,  1.4312e-01],
        [ 1.5750e-02,  3.6716e-04, -5.8441e-02,  1.1392e-01,  8.7846e-03,
         -1.1865e-01, -8.5293e-02,  2.8894e-01,  2.8925e-01,  2.0878e-01],
        [ 1.1347e-01, -4.9287e-02, -8.4584e-02,  1.8721e-01,  3.0892e-02,
         -1.8219e-01, -8.0051e-02,  3.3497e-01,  2.5591e-01,  1.7276e-01]],
       grad_fn=<AddmmBackward>)


#### nn.Softmax
신경망의 마지막 층은 범위가 [-infty, infty] 인 `logits`을 반환하므로 [nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html)을 통과시켜야 합니다. logit은 모델의 각 클래스에 대해 예측 확률을 나타내는 [0, 1] 크기의 값으로 조절됩니다. `dim` 매개변수는 해당 축의 합이 1로 유지되어야 하는 차원을 가리킵니다.

In [11]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

### Model parameters

뉴럴넷의 많은 레이어는 파라미터화되어 학습 중 최적화되는 관련 가중치와 편향을 갖게 됩니다. nn.Module을 상속한 하위 클래스는 자동으로 모델 내부에 정의된 모든 필드를 추적하고, 모델의 `parameters()` 혹은 `named_parameters()` 함수를 사용하여 모든 파라미터에 접근할 수 있도록 만듭니다.

아래 예시는 모든 파라미터를 순회하여 각 층의 사이즈와 값을 출력합니다. 

In [12]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : Parameter containing:
tensor([[ 3.1566e-02,  2.9527e-02, -2.8753e-02,  ..., -2.2680e-05,
          1.4130e-02, -1.5750e-02],
        [-1.8064e-02,  7.8598e-03, -2.8219e-02,  ..., -3.6299e-03,
          2.2727e-02, -1.3266e-02],
        [-2.2548e-02,  1.1958e-03, -4.4164e-03,  ..., -5.7172e-03,
         -3.4205e-02, -3.1153e-02],
        ...,
        [-1.4886e-02,  2.8578e-03, -3.5630e-02,  ...,  2.0857e-02,
         -6.0138e-03, -1.2595e-02],
        [-9.7030e-03,  3.3606e-02,  3.3565e-02,  ..., -2.5346e-02,
         -2.6401e-02, -1.5859e-02],
        [-1.9326e-